In [1]:
!pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
import opendatasets as od
import tensorflow as tf
import os


In [3]:
dataset = od.download(
    "https://www.kaggle.com/competitions/plant-pathology-2020-fgvc7")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: beddahana
Your Kaggle Key: ··········


100%|██████████| 779M/779M [00:12<00:00, 67.8MB/s]



Extracting archive ./plant-pathology-2020-fgvc7/plant-pathology-2020-fgvc7.zip to ./plant-pathology-2020-fgvc7


In [4]:
import torch
import torchvision
print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("CUDA is available:", torch.cuda.is_available())
import sys
!{sys.executable} -m pip install opencv-python matplotlib
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'
!mkdir images
!wget -P images https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/dog.jpg
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

PyTorch version: 2.0.1+cu118
Torchvision version: 0.15.2+cu118
CUDA is available: True
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-5mv7v5wh
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-5mv7v5wh
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36589 sha256=95f66ec612ed80604d39b7dc11fd8b55cf6d13ce7b38bee594338e9f551e21e0
  Stored in directory: /tmp/pip-ephem-wheel-cache-p92wyoet/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segm

In [5]:
def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    ax.imshow(img)


In [6]:
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam_checkpoint = "/content/sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(
    model=sam,
    points_per_side=32,
    pred_iou_thresh=0.86,
    stability_score_thresh=0.92,
    crop_n_layers=1,
    crop_n_points_downscale_factor=2,
    min_mask_region_area=100,  # Requires open-cv to run post-processing
)

In [7]:
image_dir = "/content/plant-pathology-2020-fgvc7/images"

In [8]:
output_dir = "/content/output/SAMimages"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
# Iterate over the images in the directory and apply the segment anything model to each image
import cv2
import os
import sys
import matplotlib.pyplot as plt
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
for filename in os.listdir(image_dir):
    if not filename.endswith(".jpg"):
        continue
    image_path = os.path.join(image_dir, filename)
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Generate the masks using the mask generator
    masks = mask_generator.generate(image)

   # overlay masks on the original image
    image_with_masks = image.copy()
    #for mask in masks:
     #   image_with_masks = cv2.addWeighted(image_with_masks, 1, mask, 0.5, 0)

    # save the output image
    output_filename = os.path.join(output_dir, filename)
    cv2.imwrite(output_filename, image_with_masks)
